In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image at path: {image_path}")
            return None
        image = cv2.resize(image, target_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image / 255.0
        return image
    except Exception as e:
        print(f"Exception during preprocessing image {image_path}: {e}")
        return None

def load_data(fingerprint_dir, non_fingerprint_dir, target_size=(224, 224)):
    images = []
    labels = []

    # Load fingerprint images
    for file_name in os.listdir(fingerprint_dir):
        file_path = os.path.join(fingerprint_dir, file_name)
        image = preprocess_image(file_path, target_size)
        if image is not None:
            images.append(image)
            labels.append(1)  # Label for fingerprint

    # Load non-fingerprint images
    for file_name in os.listdir(non_fingerprint_dir):
        file_path = os.path.join(non_fingerprint_dir, file_name)
        image = preprocess_image(file_path, target_size)
        if image is not None:
            images.append(image)
            labels.append(0)  # Label for non-fingerprint

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Directories for the images
fingerprint_dir = 'Fingerprints'
non_fingerprint_dir = 'non_fingerprints'


# Load and preprocess the data
images, labels = load_data(fingerprint_dir, non_fingerprint_dir)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training images shape: {train_images.shape}")
print(f"Validation images shape: {val_images.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Validation labels shape: {val_labels.shape}")


Training images shape: (1340, 224, 224, 3)
Validation images shape: (335, 224, 224, 3)
Training labels shape: (1340,)
Validation labels shape: (335,)


In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10, batch_size=32)

# Save the trained model
model.save('fingerprint_classifier_1.h5')


C:\Users\P.Leela Venkatesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 36s 817ms/step - accuracy: 0.8569 - loss: 0.3520 - val_accuracy: 0.9672 - val_loss: 0.0629
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 30s 717ms/step - accuracy: 0.9718 - loss: 0.0596 - val_accuracy: 0.9910 - val_loss: 0.0279
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 30s 704ms/step - accuracy: 0.9875 - loss: 0.0239 - val_accuracy: 0.9821 - val_loss: 0.0275
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 30s 710ms/step - accuracy: 0.9879 - loss: 0.0344 - val_accuracy: 0.9791 - val_loss: 0.0343
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 29s 701ms/step - accuracy: 0.9800 - loss: 0.0471 - val_accuracy: 0.9910 - val_loss: 0.0216
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 35s 833ms/step - accuracy: 0.9875 - loss: 0.0295 - val_accuracy: 0.9910 - val_loss: 0.0274
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 38s 913ms/step - accuracy: 0.9861 - loss: 0.0341 - val_accuracy: 0.9910 - val_loss: 0.0246
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 768ms/step - accuracy: 0.9885 - loss: 0.0272 - val_accu

In [3]:
# Load the trained model
# model = tf.keras.models.load_model('fingerprint_classifier.h5')

def classify_image(model, image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    pred=prediction[0][0]
    if pred >= 0.5:
        return 1
    else:
        return 0



In [ ]:
import os
import numpy as np 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def test_dir(directory):
    y_test=[]
    y_pred=[]
    types=os.listdir(directory)
    for t in types:
        images=os.listdir(os.path.join(directory,t))
        for image in images:
            image_path=os.path.join(os.path.join(directory,t),image)
            pred=classify_image(model,image_path)
            print(f"Predicted Output {image_path} is:{pred} ")
            y_test.append(t)
            y_pred.append(pred)

    return np.array(y_test), np.array(y_pred)

# y_test,y_pred=test_dir("fingerprint_test")
y_test,y_pred=test_dir("Blood_data")
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
print(classification_report(y_test,y_pred))

print(f'{np.round(accuracy_score(y_test,y_pred)*100,2)}% of accuracy')

print("Confusion matrix")
print(confusion_matrix(y_test,y_pred))

            
            

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted Output Blood_data\A+\aravind.jpg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Output Blood_data\A+\aravind_augmented_1.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Output Blood_data\A+\aravind_augmented_10.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Output Blood_data\A+\aravind_augmented_11.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Output Blood_data\A+\aravind_augmented_12.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Output Blood_data\A+\aravind_augmented_13.jpeg is:1 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Output Blood_data\A+\aravind_augmented_14.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Output Blood_data\A+\aravind_augmented_15.jpeg is:0 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Output Blood_data\A+\aravind_augmented_2.jpeg is:1 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Output Blood_data\A+\aravind_augmented_3.jpeg is:0 
1/1 ━━━

In [6]:
y_test,y_pred=test_dir("fingerprint_test")
y_test = y_test.astype(int)
y_pred = y_pred.astype(int)
print(classification_report(y_test,y_pred))

print(f'{np.round(accuracy_score(y_test,y_pred)*100,2)}% of accuracy')

print("Confusion matrix")
print(confusion_matrix(y_test,y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Output is:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted Output is:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Output is:  1
1/1 ━━━━

In [8]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('fingerprint_classifier.h5')

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image at path: {image_path}")
            return None
        image = cv2.resize(image, target_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image / 255.0
        return image
    except Exception as e:
        print(f"Exception during preprocessing image {image_path}: {e}")
        return None

